In [16]:
class ParameterValues:
    def __init__(self, epsnx, alfax, betax, epsny, alfay, betay, epsnz, alfaz, betaz):
        self.epsnx = epsnx
        self.alfax = alfax
        self.betax = betax
        self.epsny = epsny
        self.alfay = alfay
        self.betay = betay
        self.epsnz = epsnz
        self.alfaz = alfaz
        self.betaz = betaz

In [17]:
import subprocess
import time
import os
import shutil
import fileinput

class ParameterValues:
    def __init__(self, epsnx, alfax, betax, epsny, alfay, betay, epsnz, alfaz, betaz):
        self.epsnx = epsnx
        self.alfax = alfax
        self.betax = betax
        self.epsny = epsny
        self.alfay = alfay
        self.betay = betay
        self.epsnz = epsnz
        self.alfaz = alfaz
        self.betaz = betaz

values = ParameterValues(
    epsnx=0.1,
    alfax=-0.55,
    betax=170.0,
    epsny=0.1,
    alfay=-0.55,
    betay=170.0,
    epsnz=5.00,
    alfaz=0.10,
    betaz=10.0
)

line_numbers = {
    "epsnx": 2,
    "alfax": 3,
    "betax": 4,
    "epsny": 5,
    "alfay": 6,
    "betay": 7,
    "epsnz": 8,
    "alfaz": 9,
    "betaz": 10
}

# Update parameter values in the track.dat file
for line in fileinput.input("sim/track.dat", inplace=True):
    line_number = fileinput.lineno()
    for label, line_num in line_numbers.items():
        if line_number == line_num:
            parts = line.split("=")
            parameter_name = parts[0].strip()
            if parameter_name == label:
                new_value = str(getattr(values, label))
                line = f"{parameter_name}={new_value}d0\n"
                break
    print(line, end="")

exe_path = "sim/TRACKv39.exe"
exe_dir = os.path.dirname(exe_path)
log_file = "sim/log.out"
output_dir = os.path.join(os.getcwd(), time.strftime("%Y%m%d%H%M%S") + "_" + "_".join(f"{getattr(values, param):.2f}" for param in line_numbers.keys()))

process = subprocess.Popen(exe_path, cwd=exe_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
log_file_exists = False

while process.poll() is None or log_file_exists:
    if not log_file_exists and os.path.isfile(log_file):
        log_file_exists = True

    if log_file_exists:
        with open(log_file, "r") as file:
            lines = file.readlines()
            if len(lines) >= 8 and lines[7].strip():
                break

    time.sleep(1)

shutil.move(log_file, os.path.join(output_dir, "log.out"))
shutil.move("sim/beam.out", os.path.join(output_dir, "beam.out"))
shutil.move("sim/coord.out", os.path.join(output_dir, "coord.out"))

print("Program has terminated. Files copied to output directory:", output_dir)


AttributeError: 'NoneType' object has no attribute 'write'